In [1]:
from bs4 import BeautifulSoup as bs
import json
import jsonlines
from tqdm import tqdm

In [2]:
file = open("med.cd.az.xml", "r", encoding = "utf-8")
content = file.readlines()
bs_contents = []
for line in tqdm(content):
  bs_contents.append(bs(line, "xml"))
print(len(bs_contents))

100%|█| 44309/44309 [00:32<00:00, 1375.2

44309


In [3]:
bs_contents[2000]

<?xml version="1.0" encoding="utf-8"?>
<HOMOGRAPH ID="399281"><HEAD><BASE-FORM><ENTRY0>AT&amp;T</ENTRY0><PRONS><PRON ID="0">e ti n ti</PRON></PRONS></BASE-FORM></HEAD><SENSE><DEFINITION>American Telephone and Telegraph Company: a large US telephone company</DEFINITION><trans lang="zh_HK">美國電話電報公司</trans></SENSE></HOMOGRAPH>

In [4]:
def print_data_and_type(idx):
  i = 0
  for bs_content in tqdm(bs_contents):
    if i == idx:
      print("data:")
      print(bs_content)
      print("type:")
      print(type(bs_content))
      return
    i = i + 1

print_data_and_type(2000)

  5%| | 2000/44309 [00:00<00:00, 972817.

data:
<?xml version="1.0" encoding="utf-8"?>
<HOMOGRAPH ID="399281"><HEAD><BASE-FORM><ENTRY0>AT&amp;T</ENTRY0><PRONS><PRON ID="0">e ti n ti</PRON></PRONS></BASE-FORM></HEAD><SENSE><DEFINITION>American Telephone and Telegraph Company: a large US telephone company</DEFINITION><trans lang="zh_HK">美國電話電報公司</trans></SENSE></HOMOGRAPH>
type:
<class 'bs4.BeautifulSoup'>


# tqdm
20221013 this package is only to display current iteration progress, it does not modify the data.

In [5]:
entries = bs_contents[39].find_all("HOMOGRAPH")
print(len(entries))

1


In [6]:
entry = entries[0]
print(type(entry))

<class 'bs4.element.Tag'>


In [7]:
entry["ID"]

'398191'

In [8]:
str(entry.find_all(["ENTRY0", "ENTRY13"])[0].text)

'1922 Committee, the'

In [9]:
print('<?xml version="1.0" encoding="utf-8"?>' + '\n' + str(entry) == str(bs_contents[39]))

True


In [10]:
print(entry.find("PART-OF-SPEECH"))

None


In [11]:
data = entry.find("DEFINITION")
print("data:")
print(data)
print("type:")
print(type(data))

data:
<DEFINITION>the Conservative members of the British parliament who are <NDV>backbenchers</NDV> <GL>not ministers</GL></DEFINITION>
type:
<class 'bs4.element.Tag'>


In [12]:
print(data.get_text("DEFINITION"))

the Conservative members of the British parliament who are DEFINITIONbackbenchersDEFINITION DEFINITIONnot ministers


In [13]:
print(data.text)

the Conservative members of the British parliament who are backbenchers not ministers


In [14]:
def get_example(example):
  result = {}
  fields = ["EXAMPLE", "trans"]
  for field in fields:
    if example.find(field) != None:
      result[field] = example.find(field).text
  return result

In [15]:
def get_phrase(phrase):
  result = {}
  fields = ["DEFINITION", "MULTIWORD", "trans"]
  for field in fields:
    if phrase.find(field) != None:
      result[field] = phrase.find(field).text
  if phrase.find("EXAMPLES") != None:
    result["EXAMPLES"] = []
    for example in phrase.find_all("EXAMPLES"):
      result["EXAMPLES"].append(get_example(example))
  return result

In [16]:
def get_subsense(subsense):
  result = {}
  if subsense.find("SUB-SENSE-NUM") != None:
    result["SUB-SENSE-NUM"] = subsense.find("SUB-SENSE-NUM").text
  result.update(get_sense(subsense))
  return result

In [17]:
def get_sense(sense):
  result = {}
  fields = ["SENSE-NUM", "DEFINITION", "trans"]
  for field in fields:
    if sense.find(field) != None:
      result[field] = sense.find(field).text
  if sense.find("EXAMPLES") != None:
    result["EXAMPLES"] = []
    for example in sense.find_all("EXAMPLES"):
      result["EXAMPLES"].append(get_example(example))
  if sense.find("SUB-SENSE") != None:
    result["SUB-SENSE"] = []
    for subsense in sense.find_all("SUB-SENSE"):
      result["SUB-SENSE"].append(get_subsense(subsense))
  return result

In [18]:
def get_dict():
  result = {}
  records_num = 0
  for bs_content in tqdm(bs_contents):
    entry = bs_content.find("HOMOGRAPH") # there is <= 1 homograph per bs_content
    if entry == None:
      continue
    records_num = records_num + 1
    word_dict = {}
    word_dict["ID"] = entry["ID"]
    if entry.find("PART-OF-SPEECH") != None:
      word_dict["PART-OF-SPEECH"] = entry.find("PART-OF-SPEECH").text
    if entry.find("PHRASE") != None:
      word_dict["PHRASE"] = []
      for phrase in entry.find_all("PHRASE"):
        word_dict["PHRASE"].append(get_phrase(phrase))
    if entry.find("SENSE") != None or entry.find("SENSE45") != None:
      word_dict["SENSE"] = []
      if entry.find("SENSE") != None:
        for sense in entry.find_all("SENSE"):
          word_dict["SENSE"].append(get_sense(sense))
      if entry.find("SENSE45") != None:
        for sense in entry.find_all("SENSE45"):
          word_dict["SENSE"].append(get_sense(sense))
    result[str(entry.find_all(["ENTRY0", "ENTRY13"])[0].text)] = word_dict
  return result

my_dict = get_dict()

100%|█| 44309/44309 [00:28<00:00, 1577.0


In [19]:
print(len(my_dict))

44197


In [20]:
def print_word(key):
  print(json.dumps(my_dict[key], indent=2, ensure_ascii=False))

In [21]:
print_word("return1")

{
  "ID": "291251",
  "PART-OF-SPEECH": "verb",
  "PHRASE": [
    {
      "DEFINITION": "to say whether someone is guilty or not guilty of a crime in a court of law",
      "MULTIWORD": "return a verdict (of sth)",
      "trans": "宣佈判決",
      "EXAMPLES": [
        {
          "EXAMPLE": "After several hours the jury returned a verdict of not guilty.",
          "trans": "幾小時後陪審團宣佈無罪的判決。"
        }
      ]
    }
  ],
  "SENSE": [
    {
      "SENSE-NUM": "1",
      "DEFINITION": "to go back to a place where you were earlier, or to come back from a place where you have just been",
      "trans": "返回，回來",
      "EXAMPLES": [
        {
          "EXAMPLE": "One day she just walked out and never returned.",
          "trans": "有一天她出去後就再也沒回來。"
        },
        {
          "EXAMPLE": "Be careful. We want you to return to base in one piece.",
          "trans": "小心點。我們希望你安然無恙地返回基地。"
        },
        {
          "EXAMPLE": "And when do you return from Paris?",
          "trans": "你何時從巴黎回來？

In [22]:
print_word("record2")

{
  "ID": "284722",
  "PART-OF-SPEECH": "verb",
  "SENSE": [
    {
      "SENSE-NUM": "1",
      "DEFINITION": "to make a record of something that has happened, usually by writing it down",
      "trans": "記錄",
      "EXAMPLES": [
        {
          "EXAMPLE": "They were asked to record the time at which the attack happened.",
          "trans": "他們被要求記錄下襲擊發生的時間。"
        },
        {
          "EXAMPLE": "The coroner recorded a verdict of suicide.",
          "trans": "法醫記錄下自殺的判定。"
        },
        {
          "EXAMPLE": "His diaries record the lives of ordinary country people.",
          "trans": "他的日記記載了普通鄉下人的生活。"
        },
        {
          "EXAMPLE": "Temperatures as low as –70 degrees Celsius have been recorded.",
          "trans": "儀器顯示溫度曾低至攝氏70度。"
        }
      ],
      "SUB-SENSE": [
        {
          "SUB-SENSE-NUM": "1a",
          "DEFINITION": "to officially decide something and write it down",
          "trans": "（正式地）記錄",
          "EXAMPLES": [
            {

In [23]:
print_word("recover1")

{
  "ID": "284821",
  "PART-OF-SPEECH": "verb",
  "SENSE": [
    {
      "SENSE-NUM": "1",
      "DEFINITION": "to become fit and healthy again after an illness or injury",
      "trans": "痊癒；康復",
      "EXAMPLES": [
        {
          "EXAMPLE": "I havent fully recovered from that flu I had.",
          "trans": "我所患的流感還沒痊癒。"
        },
        {
          "EXAMPLE": "He was fired last month and hes still recovering from the shock.",
          "trans": "他上個月被僱了，至今尚未從這次打擊中恢復過來。"
        },
        {
          "EXAMPLE": "The housing market appears to be recovering from the recession.",
          "trans": "房地産市場好像正從不景氣中復甦。"
        }
      ],
      "SUB-SENSE": [
        {
          "SUB-SENSE-NUM": "1a",
          "DEFINITION": "to stop being affected by an unpleasant experience",
          "trans": "（從不愉快的經歷中）恢復",
          "EXAMPLES": [
            {
              "EXAMPLE": "He was fired last month and hes still recovering from the shock.",
              "trans": "他上個月被僱了，至今尚未從這次

### Output example
`my_dict['record1']`
```json
{
    'ID': '284721',
    'PART-OF-SPEECH': 'noun',
    'PHRASE': [
               {'DEFINITION': '...',
                'EXAMPLES': ...,
                'MULTIWORD': '...',
                'TRANSLATION': '...'},
                ...
               ],
            
     'SENSE': [
               {'DEFINITION': '...',
                'EXAMPLES': ...,
                'SUB-SENSE': [{'DEFINITION': '...',
                               'EXAMPLES': ...,
                               'TRANSLATION': '...'}],
                'TRANSLATION': '...'},
                ...
               ]
}
```